In [ ]:
# # Extract text from PDF
# pdf_path = 'OTC_TATLY_2022.pdf'
# extracted_text = extract_text(pdf_path)

# # Save to text file
# output_file = 'OTC_TATLY_2022.txt'
# with open(output_file, 'w', encoding='utf-8') as f:
#     f.write(extracted_text)

# print(f"Text extracted and saved to {output_file}")
# print(f"Total characters: {len(extracted_text)}")

Text extracted and saved to OTC_TATLY_2022.txt
Total characters: 1680500


In [5]:
import pdfplumber
import re
from pathlib import Path

In [10]:
import pdfplumber
from pathlib import Path

def extract_raw_text(pdf_path, output_txt_path):
    pdf_path = Path(pdf_path)
    output_txt_path = Path(output_txt_path)

    all_text = []
    extracted_pages = 0

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            # Use tolerant extraction for complex layouts
            text = page.extract_text(
                x_tolerance=2,
                y_tolerance=2
            )

            if text and text.strip():
                all_text.append(text)
                extracted_pages += 1
            else:
                # Diagnostic print (very useful for debugging)
                print(f"⚠️ Page {i}: no extractable text")

    final_text = "\n\n".join(all_text)

    # Ensure output directory exists
    output_txt_path.parent.mkdir(parents=True, exist_ok=True)

    # Save extracted text
    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write(final_text)

    print("✅ RAW TEXT EXTRACTION COMPLETE")
    print(f"📄 PDF: {pdf_path.name}")
    print(f"📄 Output: {output_txt_path}")
    print(f"📑 Pages with text: {extracted_pages}/{len(pdf.pages)}")
    print(f"🔢 Total characters: {len(final_text)}")

    return final_text


In [ ]:
# pdf_path = "./dataset/raw_pdfs/OTC_TATLY_2022.pdf"
# output_path = "./dataset/extracted_text/OTC_TATLY_2022_structured.txt"

# text = extract_raw_text(pdf_path, output_path)


⚠️ Page 1: no extractable text
⚠️ Page 475: no extractable text
✅ RAW TEXT EXTRACTION COMPLETE
📄 PDF: OTC_TATLY_2022.pdf
📄 Output: dataset\extracted_text\OTC_TATLY_2022_structured.txt
📑 Pages with text: 473/475
🔢 Total characters: 1680449


## Function to Actually extract the Text using stop keywords comditions

In [ ]:
import pdfplumber
import re
from tqdm import tqdm
from pathlib import Path

# ===================== CONFIG =====================

INCLUDE_KEYWORDS = [
    "sustainability", "esg", "environment", "climate",
    "value creation", "materiality", "stakeholder",
    "strategy", "vision", "mission", "csr", "governance"
    "biodiversity", "water", "emissions", "net zero"
]

STOP_KEYWORDS = [
    "financial statements",
    "balance sheet",
    "profit and loss",
    "cash flow",
    "board of directors",
    "notice of agm",
    "statutory reports"
]

MIN_PAGE_TEXT_LENGTH = 300   # Ignore junk pages

# ===================== HELPERS =====================

def clean_text(text: str) -> str:
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'page\s+\d+', '', text, flags=re.I)
    return text.strip()

def contains_any(text, keywords):
    text = text.lower()
    return any(k in text for k in keywords)

# ===================== MAIN FUNCTION =====================

def extract_relevant_sections_safe(pdf_path, output_path):
    pdf_path = Path(pdf_path)
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)

    collected_text = []

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(tqdm(pdf.pages, desc="Processing pages")):
            try:
                text = page.extract_text(x_tolerance=2, y_tolerance=2)
            except Exception:
                continue

            if not text:
                continue

            text = clean_text(text)

            # Skip very small text pages
            if len(text) < MIN_PAGE_TEXT_LENGTH:
                continue

            # Drop pages that contain STOP keywords
            if contains_any(text, STOP_KEYWORDS):
                continue

            # Keep only relevant pages
            if contains_any(text, INCLUDE_KEYWORDS):
                collected_text.append(f"\n--- PAGE {page_num + 1} ---\n{text}")

    final_text = "\n".join(collected_text)

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(final_text)

    print("✅ Extraction complete")
    print(f"📄 Pages extracted: {len(collected_text)}")
    print(f"🧠 Characters extracted: {len(final_text)}")

    return final_text


In [24]:
pdf_path = "./dataset/raw_pdfs/OTC_TATLY_2022.pdf"
output_path = "./dataset/extracted_text/OTC_TATLY_2022_esg.txt"

text = extract_relevant_sections_safe(pdf_path, output_path)


Processing pages: 100%|██████████| 475/475 [02:09<00:00,  3.66it/s]

✅ Extraction complete
📄 Pages extracted: 144
🧠 Characters extracted: 622245


In [25]:
pdf_path = "./dataset/raw_pdfs/OTC_TATLY_2021.pdf"
output_path = "./dataset/extracted_text/OTC_TATLY_2021_esg.txt"

text = extract_relevant_sections_safe(pdf_path, output_path)

Processing pages: 100%|██████████| 233/233 [02:32<00:00,  1.52it/s]

✅ Extraction complete
📄 Pages extracted: 82
🧠 Characters extracted: 454870


### Function For to run whole batch of raw_data pdfs

In [26]:
from pathlib import Path

def batch_extract_esg_pdfs(
    input_dir="./dataset/raw_pdf",
    output_dir="./dataset/extracted_text"
):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    summary = []

    pdf_files = list(input_dir.glob("*.pdf"))

    print(f"📂 Found {len(pdf_files)} PDF files\n")

    for pdf_file in pdf_files:
        try:
            output_file = output_dir / f"{pdf_file.stem}_esg.txt"

            print(f"▶️ Processing: {pdf_file.name}")
            text = extract_relevant_sections_safe(pdf_file, output_file)

            summary.append({
                "file": pdf_file.name,
                "pages": len(text) // 4000,   # rough estimate
                "characters": len(text)
            })

        except Exception as e:
            print(f"❌ Failed: {pdf_file.name}")
            print(str(e))
            summary.append({
                "file": pdf_file.name,
                "pages": 0,
                "characters": 0,
                "error": str(e)
            })

    print("\n✅ Batch extraction complete\n")

    for s in summary:
        print(s)

    return summary
batch_extract_esg_pdfs()

📂 Found 2 PDF files

▶️ Processing: NYSE_VEDL_2021.pdf


Processing pages: 100%|██████████| 132/132 [00:44<00:00,  2.97it/s]


✅ Extraction complete
📄 Pages extracted: 99
🧠 Characters extracted: 238045
▶️ Processing: OTC_TATLY_2024.pdf


Processing pages: 100%|██████████| 581/581 [03:00<00:00,  3.22it/s]

✅ Extraction complete
📄 Pages extracted: 134
🧠 Characters extracted: 570600

✅ Batch extraction complete

{'file': 'NYSE_VEDL_2021.pdf', 'pages': 59, 'characters': 238045}
{'file': 'OTC_TATLY_2024.pdf', 'pages': 142, 'characters': 570600}


[{'file': 'NYSE_VEDL_2021.pdf', 'pages': 59, 'characters': 238045},
 {'file': 'OTC_TATLY_2024.pdf', 'pages': 142, 'characters': 570600}]

In [27]:
batch_extract_esg_pdfs()

📂 Found 2 PDF files

▶️ Processing: NYSE_VEDL_2023.pdf


Processing pages: 100%|██████████| 148/148 [00:41<00:00,  3.53it/s]


✅ Extraction complete
📄 Pages extracted: 138
🧠 Characters extracted: 420297
▶️ Processing: NYSE_VEDL_2024.pdf


Processing pages: 100%|██████████| 119/119 [00:42<00:00,  2.82it/s]

✅ Extraction complete
📄 Pages extracted: 110
🧠 Characters extracted: 463684

✅ Batch extraction complete

{'file': 'NYSE_VEDL_2023.pdf', 'pages': 105, 'characters': 420297}
{'file': 'NYSE_VEDL_2024.pdf', 'pages': 115, 'characters': 463684}


[{'file': 'NYSE_VEDL_2023.pdf', 'pages': 105, 'characters': 420297},
 {'file': 'NYSE_VEDL_2024.pdf', 'pages': 115, 'characters': 463684}]

In [28]:
batch_extract_esg_pdfs()

📂 Found 16 PDF files

▶️ Processing: american-forest-paper-association_2016.pdf


Processing pages: 100%|██████████| 32/32 [00:02<00:00, 13.74it/s]


✅ Extraction complete
📄 Pages extracted: 20
🧠 Characters extracted: 34108
▶️ Processing: american-forest-paper-association_2018.pdf


Processing pages: 100%|██████████| 16/16 [00:25<00:00,  1.57s/it]


✅ Extraction complete
📄 Pages extracted: 13
🧠 Characters extracted: 31021
▶️ Processing: american-forest-paper-association_2020.pdf


...

Processing pages: 100%|██████████| 128/128 [00:28<00:00,  4.42it/s]


✅ Extraction complete
📄 Pages extracted: 116
🧠 Characters extracted: 350457
▶️ Processing: NYSE_MUR_2024.pdf


Processing pages: 100%|██████████| 112/112 [00:27<00:00,  4.12it/s]

✅ Extraction complete
📄 Pages extracted: 98
🧠 Characters extracted: 322384

✅ Batch extraction complete

{'file': 'american-forest-paper-association_2016.pdf', 'pages': 8, 'characters': 34108}
{'file': 'american-forest-paper-association_2018.pdf', 'pages': 7, 'characters': 31021}
{'file': 'american-forest-paper-association_2020.pdf', 'pages': 10, 'characters': 42718}
{'file': 'LSE_RIGD_2021.pdf', 'pages': 105, 'characters': 423908}
{'file': 'LSE_RIGD_2022.pdf', 'pages': 158, 'characters': 634703}
{'file': 'LSE_RIGD_2023.pdf', 'pages': 39, 'characters': 156434}
{'file': 'LSE_RIGD_2024.pdf', 'pages': 46, 'characters': 186068}
{'file': 'NASDAQ_NVDA_2022.pdf', 'pages': 30, 'characters': 121798}
{'file': 'NASDAQ_NVDA_2023.pdf', 'pages': 38, 'characters': 152173}
{'file': 'NASDAQ_NVDA_2024.pdf', 'pages': 24, 'characters': 97337}
{'file': 'NYSE_BA_2021.pdf', 'pages': 44, 'characters': 178920}
{'file': 'NYSE_BA_2022.pdf', 'pages': 55, 'characters': 220720}
{'file': 'NYSE_MUR_2021.pdf', 'pages'

[{'file': 'american-forest-paper-association_2016.pdf',
  'pages': 8,
  'characters': 34108},
 {'file': 'american-forest-paper-association_2018.pdf',
  'pages': 7,
  'characters': 31021},
 {'file': 'american-forest-paper-association_2020.pdf',
  'pages': 10,
  'characters': 42718},
 {'file': 'LSE_RIGD_2021.pdf', 'pages': 105, 'characters': 423908},
 {'file': 'LSE_RIGD_2022.pdf', 'pages': 158, 'characters': 634703},
 {'file': 'LSE_RIGD_2023.pdf', 'pages': 39, 'characters': 156434},
 {'file': 'LSE_RIGD_2024.pdf', 'pages': 46, 'characters': 186068},
 {'file': 'NASDAQ_NVDA_2022.pdf', 'pages': 30, 'characters': 121798},
 {'file': 'NASDAQ_NVDA_2023.pdf', 'pages': 38, 'characters': 152173},
 {'file': 'NASDAQ_NVDA_2024.pdf', 'pages': 24, 'characters': 97337},
 {'file': 'NYSE_BA_2021.pdf', 'pages': 44, 'characters': 178920},
 {'file': 'NYSE_BA_2022.pdf', 'pages': 55, 'characters': 220720},
 {'file': 'NYSE_MUR_2021.pdf', 'pages': 51, 'characters': 205318},
 {'file': 'NYSE_MUR_2022.pdf', 'pages':

In [29]:
pdf_path = "./dataset/raw_pdfs/NYSE_CRI_2020.pdf"
output_path = "./dataset/extracted_text/NYSE_CRI_2020_esg.txt"

text = extract_relevant_sections_safe(pdf_path, output_path)


Processing pages: 100%|██████████| 46/46 [00:07<00:00,  5.98it/s]

✅ Extraction complete
📄 Pages extracted: 25
🧠 Characters extracted: 59775


In [30]:
pdf_path = "./dataset/raw_pdfs/NYSE_CRI_2021.pdf"
output_path = "./dataset/extracted_text/NYSE_CRI_2021_esg.txt"

text = extract_relevant_sections_safe(pdf_path, output_path)


Processing pages: 100%|██████████| 63/63 [00:10<00:00,  6.18it/s]

✅ Extraction complete
📄 Pages extracted: 34
🧠 Characters extracted: 87580


In [31]:
pdf_path = "./dataset/raw_pdfs/NYSE_CRI_2022.pdf"
output_path = "./dataset/extracted_text/NYSE_CRI_2022_esg.txt"

text = extract_relevant_sections_safe(pdf_path, output_path)


Processing pages: 100%|██████████| 71/71 [00:18<00:00,  3.78it/s]

✅ Extraction complete
📄 Pages extracted: 40
🧠 Characters extracted: 100456
